In [1]:
#from sklearn.externals import joblib
from tensorflow.keras.models import load_model

import librosa
import librosa.display
import IPython.display
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as fm
%matplotlib inline

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pwd

/content


In [9]:
%cd drive/My Drive/Colab_Notebooks/Music Mood Recognition

/content/drive/My Drive/Colab_Notebooks/Music Mood Recognition


In [10]:
!ls

1_Scraping.ipynb			    funcs.ipynb
2_EDA					    funcs.py
3_Modeling_with_preexisting_features.ipynb  GenreTraniningModel.h5
4_feature_engineering.ipynb		    major-scale.mid
5_modeling-with-extracted-features.ipynb    midi_to_wav.ipynb
6_Play.ipynb				    output.mp3
auth.py					    __pycache__
data					    scrapes
EDA.py					    Scraping.py
font.sf2				    test.mp3


In [6]:
pip install python_speech_features

  Created wheel for python-speech-features: filename=python_speech_features-0.6-py3-none-any.whl size=5888 sha256=5e748da3859fbc5e345237eb7f269ae71a8df284dd06d017ef4e4a30d7626ebb
  Stored in directory: /root/.cache/pip/wheels/b0/0e/94/28cd6afa3cd5998a63eef99fe31777acd7d758f59cf24839eb
Successfully built python-speech-features


In [7]:
pip install spotipy

     |████████████████████████████████| 63 kB 1.3 MB/s 
     |████████████████████████████████| 138 kB 18.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [12]:
import os
import pandas as pd
import numpy as np
import librosa, librosa.display
import matplotlib.pyplot as plt
import matplotlib.style as style
import seaborn as sns
import regex
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from ast import literal_eval
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from funcs import *

In [13]:
audio_path = 'output.mp3'
x, sr = librosa.load(audio_path, mono=True, duration=30 )
# ori_sent = '그는 괜찮은 척 하려고 애쓰는 것 같았다'


In [14]:
IPython.display.Audio(data=x, rate=sr)

In [15]:
model = load_model('GenreTraniningModel.h5')

In [16]:
#conver Audio to Mono
x = librosa.to_mono(x)

#Normalize the raw audio
x = librosa.util.normalize(x)
hop_length = 256
frame_length = 512
n_mfcc=26


energy_calc = calc_energy(x)
rms_calc = calc_rms(x)
mfcc_calc = calc_mfcc(x, sr, n_mfcc)

mean_energy= energy_calc.mean() #progress_apply(lambda x: x.mean())
std_energy = energy_calc.std() #progress_apply(lambda x: x.std())
min_energy = energy_calc.min() #progress_apply(lambda x: x.min())
max_energy = energy_calc.max() #progress_apply(lambda x: x.max())
mfcc_mean = mfcc_calc.mean(axis=0) #apply(lambda x: x.mean(axis=0))
mfcc_std  = mfcc_calc.std(axis=0) #apply(lambda x: x.std(axis=0))
mfcc_min  = mfcc_calc.min(axis=0) #apply(lambda x: x.min(axis=0))
mfcc_max  = mfcc_calc.max(axis=0) #apply(lambda x: x.max(axis=0))
max_rms = np.max(rms_calc) #apply(np.max)
std_rms = np.std(rms_calc) #apply(np.std)
median_rms = np.median(rms_calc) #apply(np.median)
min_rms = np.min(rms_calc) #apply(np.min)

In [17]:
mfcc_mean = mfcc_mean.tolist()
mfcc_std = mfcc_std.tolist()
mfcc_min = mfcc_min.tolist()
mfcc_max = mfcc_max.tolist()

In [18]:
print(mfcc_mean)

[-160.16128540039062, 142.32492065429688, -58.91773223876953, 54.76082992553711, 24.769948959350586, 17.195661544799805, 1.7644429206848145, -1.3988150358200073, 0.95693439245224, -9.670366287231445, -4.562759876251221, 1.246179461479187, -0.13049250841140747, 1.4507334232330322, -0.4185044467449188, -7.239775657653809, 5.242345333099365, 6.10038423538208, -5.796902179718018, -4.2679948806762695, 0.7159140706062317, 1.6319653987884521, 0.12925361096858978, 1.0044745206832886, -4.409313201904297, -2.560621976852417]


In [23]:
#mfcc = librosa.feature.mfcc(x=x, sr=sr)
#
buffer = []
labels = ['Dark','Happy','Relaxing','Romance','Sad'] #조사하기

In [24]:
from collections.abc import Iterable
# https://m.blog.naver.com/hankrah/221929249131 참조 : 예제 9
def deep_flatten(lst):
  return [a for i in lst for a in deep_flatten(i)] if isinstance(lst,Iterable) else [lst]

In [25]:
#for element in mfcc:
buffer.extend([min_energy,max_energy,max_rms,std_rms,median_rms,min_rms,mfcc_mean,mfcc_std,mfcc_max,mfcc_min]) # list 안에 list 모두 flatten, input 맞추기
x_test = deep_flatten(buffer) #np.array([buffer]) # 다시 2차원 배열로 만든다.
x_test = np.array([x_test])
#print(x_test) # https://wikidocs.net/71755 model.predict 참조

Happy


In [28]:
y_predict = model.predict(x_test)

#print(y_predict) #[[3.3286267e-21 9.8352991e-02 2.0311566e-15 1.0989858e-11 9.0164697e-01]]
#print(y_predict.argmax(axis=1)) # [4]
#print(y_predict.argmax(axis=1)[0]) # 4
print(labels[y_predict.argmax(axis=1)[0]]) # Sad

Happy


In [42]:
import random
import os

path = "/content/drive/MyDrive/Colab_Notebooks/Music Mood Recognition/scrapes/"
files=os.listdir(path)


for folder in files :
  if folder == labels[y_predict.argmax(axis=1)[0]] :
    path += folder
    files=os.listdir(path)
    d=random.choice(files)
    path+=str("/"+str(d))
    print(d)
    break
x, sr = librosa.load(path, mono=True, duration=30)
IPython.display.Audio(data=x, rate=sr)


0035_Sinfonia in G Major RV 149 III Allegro.mp3


/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
